All parameters are seperated by a "&" character

Place Search Request:  
https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters

parameters wew ill be using:
- radius = 16200 (meters, approx 10 miles) 
- keyword 
- pagetoken  (gained from next_page_token)
- key  (api_key)
- location (lat, lng formatted as: "lat%2Clng")



Place Details Request:  
https://maps.googleapis.com/maps/api/place/details/json?parameters


parameters we will be using:
- place_id


No more than 60 results will be returned for each query. mutiple locations will have to be used for this search.

GPS Locations to use 
- 44.273518, -123.271058  :  Just North of Junction City
- 43.911540, -123.129609  :  Just South of Eugene 
- 44.135766, -123.345221  :  West of Eugene
- 44.046996, -123.102148  :  Eugene
- 45.669287, -121.538651  :  Near Hood River
- 44.823046, -123.043092  :  South of Salem
- 45.089875, -123.085669  :  North of Salem
- 45.110889, -123.317063  :  NW of Salem
- 44.986040, -123.286169  :  West of Salem
- 45.269130, -123.209947  :  N of McMinville
- 45.296668, -123.052705  :  Dundee
- 45.468841, -123.163255  :  Forest Grove
- 45.428859, -122.921556  :  Farmington
- 45.724679, -119.215730  :  Echo
- 45.916807, -118.421968  :  Milton-Freewater
- 44.817970, -117.647432  :  Baker City
- 44.376467, -121.260561  :  Madras
- 44.114786, -121.369051  :  Bend
- 43.214300, -123.467444  :  West Roseburg
- 43.355756, -123.424872  :  North Roseburg
- 43.468983, -123.240851  :  East Roseburg
- 43.631220, -123.523062  :  Kellogg
- 43.775670, -123.039664  :  Walden

In [1]:
import pandas as pd
import numpy as np
import os
import requests
from config import gkey
from time import sleep

nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
details_url = "https://maps.googleapis.com/maps/api/place/details/json?"


In [2]:
# payload is a dictionary that holds the parameters and their values

payload = {"keyword":"Vineyards", 
           'radius':"20000", "key":gkey,
           "location":"45.632858,-121.511857"}



In [3]:
response = requests.get(nearby_url, params=payload).json()
response

{'html_attributions': [],
 'next_page_token': 'AcYSjRiqBCaON3YSkQgIfPVAIKNCBo1QSAwHvYL_FHNSnrwFHPElY9q0w9qcFvAD2Hq8oCDHGv808mGuwdHx2OdfRJ-gTuFq5DtSYRIqjYuAXFzTmNCN0-oBk3lT8gfrmITMcQxlZrusDWABTWTeJ-0dCVfhX-ZqPYXuyliAj3M1whFvRFkucpusadNKaSh-p2R9shcDXEh54YzNMIaJ2K_B6D1EmhIHEdkBi_zrYTzFFWIomLsMcv5oWJiWWE1QWwYt_CchNRtuctwiEAnh9T_IkojxWWddDidooSIonFAoUdODB8ML1OqQgnUX66Y2-HhhAMG0sAm-DDx2QXM9WW4HEBW8GU16g16FKAUyLIIESAzJMGkqqmB76R93_L-PDNFtDX43xqY3WhSgM4Oz0rhGdVmFFK_R5PjfE539kZMqT3RQjNlrwotaOrRV_OQ',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 45.6922382, 'lng': -121.5518853},
    'viewport': {'northeast': {'lat': 45.69394342989272,
      'lng': -121.5505312701072},
     'southwest': {'lat': 45.69124377010728, 'lng': -121.5532309298927}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/gen

In [4]:
response.keys()

dict_keys(['html_attributions', 'next_page_token', 'results', 'status'])

In [5]:
response["results"]

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 45.6922382, 'lng': -121.5518853},
   'viewport': {'northeast': {'lat': 45.69394342989272,
     'lng': -121.5505312701072},
    'southwest': {'lat': 45.69124377010728, 'lng': -121.5532309298927}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
  'name': 'Marchesi Vineyards & Winery',
  'opening_hours': {'open_now': False},
  'photos': [{'height': 3024,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113821960352027495811">A Google User</a>'],
    'photo_reference': 'AcYSjRjnesT4IlxnO3C4WwCpswFKafAvuUaeeTtBm63RE6aASDYYUbgGIW7KKqYvwweCA439AzbVfYVleeUpz8l4bdSQPhFgbbJaCMu76s3QkzTnDGjn0bYpNixkIKVQCS0j95RMBVDYwYU4oe-ZDAF546Zxdrwd5YmVc5MfSjEJYDse1vfh',
    'width': 4032}],
  'place_id': 'ChIJbe-ki78OllQREZ9vEzd7Row',
 

In [6]:
# save the place_id from each result to use for the place detail query
[place["place_id"] for place in response["results"]]
            

['ChIJbe-ki78OllQREZ9vEzd7Row',
 'ChIJT8hzlUwHllQRi9bkzj6LCk0',
 'ChIJG1BslP0FllQRMRc0KTybvYM',
 'ChIJc3SZKa8OllQRJecy6f4_aXQ',
 'ChIJk8AyJCMJllQR5nx396gJhgc',
 'ChIJp6xxvqIOllQRNiNg9-9Uef4',
 'ChIJXSTLXEH9lVQRtUsfz9kHtr4',
 'ChIJnWx_ltkNllQRk6RidCeS5gc',
 'ChIJg4dXBoIallQR_zn_01-z-h0',
 'ChIJ18LXgwARllQRCm50jM-wEfs',
 'ChIJGV_eUG8OllQRPmcuXsJM5iw',
 'ChIJlzpaZkMPllQR4HWKqYGmRk4',
 'ChIJY4qS1W4OllQRvAyS2AuKE98',
 'ChIJc1og-nwQllQREipN_ZWqFZI',
 'ChIJ_UakGWQMllQRNgiEOSHJE6I',
 'ChIJgb7K8roNllQRS2Cz4_O6o2Q',
 'ChIJp3eA550OllQR5gujRSmfdmA',
 'ChIJg-OAKzYPllQRo25JEjZ9WPs',
 'ChIJbeIl22ANllQRsR4Yt5JcK8U',
 'ChIJ5ZhAntIRllQRI-sBmQwxJUI']

In [7]:
requests.get(nearby_url+f"key={gkey}&pagetoken={response['next_page_token']}").json()

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [8]:
areas_to_search = ["44.273518,-123.271058", # Just North of Junction City
"43.911540,-123.129609", # Just South of Eugene 
"44.135766,-123.345221", # West of Eugene
"44.231045,-123.311644", # West of Junction City
"44.385577,-123.330183", # Notrh of Junction City                   
"45.669287,-121.538651", # Near Hood River
"44.823046,-123.043092", # South of Salem
"45.089875,-123.085669", # North of Salem
"45.110889,-123.317063", # NW of Salem
"44.986040,-123.286169", # West of Salem
"45.228954,-123.207274", # McMinville
"45.265211,-123.028746", # Dundee
"45.346827,-123.171912", # Yamhill
"45.375533,-123.007460", # N of Newberg
"45.324865,-122.869444", # South of Newberg
"45.468841,-123.163255", # Forest Grove
"45.428859,-122.921556", # Farmington
"45.724679,-119.215730", # Echo
"45.953885, -118.451395", # West Milton-Freewater
"45.900631, -118.334322", # South Milton-Freewater
"45.947440, -118.304110", # East Milton-Freewater
"46.006372, -118.342562", # Nort Milton-Freewater
"44.817970,-117.647432", # Baker City
"44.376467,-121.260561", # Madras
"44.114786,-121.369051", # Bend
"43.214300,-123.467444", # West Roseburg
"43.355756,-123.424872", # North Roseburg
"43.468983,-123.240851", # East Roseburg
"43.631220,-123.523062", # Kellogg
"43.775670,-123.039664", # Walden
"45.632858,-121.511857"] # Hood River  

In [9]:
## Looping Through all result pages of one search grabing the next_page_tokens and place_id's

## Global Variables requied for the loops
saved_place_ids = set()
page_token_list = []

more_results = True


while more_results:    
    if len(page_token_list) == 0:
        response = requests.get(nearby_url, payload).json() 
    else:
        url = nearby_url+ f"key={gkey}&pagetoken={page_token_list[-1]}"
        response = requests.get(url).json() 

    for place in response["results"]:
        saved_place_ids.add(place["place_id"])
    try:
        page_token_list.append(response["next_page_token"])
    except:
        more_results = False 

## giving the api time to activate the next page token and prevent an invalid response
    sleep(2)
    

print("Number of page_tokens saved:", len(page_token_list))
print("Number of unique place_ids saved:", len(saved_place_ids))
print("==============\n===============\n===============")
    

Number of page_tokens saved: 2
Number of unique place_ids saved: 46


In [10]:
# Looping through the locations and performing a search for each one, grabbing the pagetokens and place_ids

## Global Variables requied for the loops
saved_place_ids = set()


for i, area in enumerate(areas_to_search):
    payload["location"] = area
    more_results = True
    page_token_list = []

## Looping Through all result pages of one search grabing the next_page_tokens and place_id's
    while more_results:    
        if len(page_token_list) == 0:
            response = requests.get(nearby_url, payload).json() 
        else:
            url = nearby_url+ f"key={gkey}&pagetoken={page_token_list[-1]}"
            response = requests.get(url).json() 

        for place in response["results"]:
            saved_place_ids.add(place["place_id"])
        try:
            page_token_list.append(response["next_page_token"])
        except:
            more_results = False 
            

        sleep(2)
    
    print(f"{i}. End for this location, moving onto the next. \nNew Totals:")    
    print("Number of page_tokens saved this time:", len(page_token_list))
    print("Number of unique place_ids saved:", len(saved_place_ids))
    print("==============\n===============\n===============")

## radius:20000 = 405 results    

0. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 22
1. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 48
2. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 48
3. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 48
4. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 0
Number of unique place_ids saved: 52
5. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 97
6. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 135
7. End for this location, 

In [11]:
# Looping through the locations and performing a search for each one, grabbing the pagetokens and place_ids
# Change the radius to attempt more results.
## Global Variables requied for the loops
saved_place_ids = set()
payload["radius"] = "50000"

for i, area in enumerate(areas_to_search):
    payload["location"] = area
    more_results = True
    page_token_list = []

## Looping Through all result pages of one search grabing the next_page_tokens and place_id's
    while more_results:    
        if len(page_token_list) == 0:
            response = requests.get(nearby_url, payload).json() 
        else:
            url = nearby_url+ f"key={gkey}&pagetoken={page_token_list[-1]}"
            response = requests.get(url).json() 

        for place in response["results"]:
            saved_place_ids.add(place["place_id"])
        try:
            page_token_list.append(response["next_page_token"])
        except:
            more_results = False 
            

        sleep(2)
    
    print(f"{i}. End for this location, moving onto the next. \nNew Totals:")    
    print("Number of page_tokens saved this time:", len(page_token_list))
    print("Number of unique place_ids saved:", len(saved_place_ids))
    print("==============\n===============\n===============")

## radius:50000 = 345 results

0. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 53
1. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 66
2. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 66
3. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 66
4. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 78
5. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 134
6. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 189
7. End for this location,

In [12]:
# Looping through the locations and performing a search for each one, grabbing the pagetokens and place_ids
# Change the radius to attempt more results.

## Global Variables requied for the loops
saved_place_ids = set()
payload["radius"] = "30000"


for i, area in enumerate(areas_to_search):
    payload["location"] = area
    more_results = True
    page_token_list = []

## Looping Through all result pages of one search grabing the next_page_tokens and place_id's
    while more_results:    
        if len(page_token_list) == 0:
            response = requests.get(nearby_url, payload).json() 
        else:
            url = nearby_url+ f"key={gkey}&pagetoken={page_token_list[-1]}"
            response = requests.get(url).json() 

        for place in response["results"]:
            saved_place_ids.add(place["place_id"])
        try:
            page_token_list.append(response["next_page_token"])
        except:
            more_results = False 
            

        sleep(2)
    
    print(f"{i}. End for this location, moving onto the next. \nNew Totals:")    
    print("Number of page_tokens saved this time:", len(page_token_list))
    print("Number of unique place_ids saved:", len(saved_place_ids))
    print("==============\n===============\n===============")
## radius:30000 = 372 results

0. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 37
1. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 50
2. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 51
3. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 51
4. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 58
5. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 113
6. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 171
7. End for this location,

## Status update
- With our gps locations, a radius of 20km, 30km, and 50km were tested. 20km has the highest number of unique reuslts.
- Time to start exploring the detailed data pull from google.


In [13]:
# detail_response = requests.get(details_url+f"place_id={saved_place_ids[0]}")
first_place = saved_place_ids.pop()
first_place

'ChIJ47fROEcPwFQRmvkED1tgYu8'

In [14]:
detail_response = requests.get(details_url+f"place_id={first_place}&key={gkey}").json()
detail_response

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'A',
    'short_name': 'A',
    'types': ['subpremise']},
   {'long_name': '189', 'short_name': '189', 'types': ['street_number']},
   {'long_name': 'Ankeny Hill Road Southeast',
    'short_name': 'Ankeny Hill Rd SE',
    'types': ['route']},
   {'long_name': 'Jefferson',
    'short_name': 'Jefferson',
    'types': ['locality', 'political']},
   {'long_name': 'Marion County',
    'short_name': 'Marion County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Oregon',
    'short_name': 'OR',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '97352', 'short_name': '97352', 'types': ['postal_code']}],
  'adr_address': '<span class="street-address">189 Ankeny Hill Rd SE A</span>, <span class="locality">Jefferson</span>, <span class="region">OR</span> <span class="p

In [15]:
detail_response.keys()

dict_keys(['html_attributions', 'result', 'status'])

In [16]:
detail_response["result"].keys()

dict_keys(['address_components', 'adr_address', 'business_status', 'delivery', 'formatted_address', 'formatted_phone_number', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'international_phone_number', 'name', 'photos', 'place_id', 'plus_code', 'rating', 'reference', 'reviews', 'types', 'url', 'user_ratings_total', 'utc_offset', 'vicinity', 'website'])

In [17]:
detail_response["result"]

{'address_components': [{'long_name': 'A',
   'short_name': 'A',
   'types': ['subpremise']},
  {'long_name': '189', 'short_name': '189', 'types': ['street_number']},
  {'long_name': 'Ankeny Hill Road Southeast',
   'short_name': 'Ankeny Hill Rd SE',
   'types': ['route']},
  {'long_name': 'Jefferson',
   'short_name': 'Jefferson',
   'types': ['locality', 'political']},
  {'long_name': 'Marion County',
   'short_name': 'Marion County',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Oregon',
   'short_name': 'OR',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'United States',
   'short_name': 'US',
   'types': ['country', 'political']},
  {'long_name': '97352', 'short_name': '97352', 'types': ['postal_code']}],
 'adr_address': '<span class="street-address">189 Ankeny Hill Rd SE A</span>, <span class="locality">Jefferson</span>, <span class="region">OR</span> <span class="postal-code">97352</span>, <span class="country-name">USA<

In [18]:
detail_list=[first_place]  ## trash varibale to test the api calling multiple times 
for x in range(3):
    detail_list.append(saved_place_ids.pop())  ## takes the id out of saved_place_id's and puts it in detail list

In [19]:
for place in detail_list:
    print(requests.get(details_url+f"key={gkey}&place_id={place}").json()["result"]["name"])
    saved_place_ids.add(place) ## puts the id back into the saved_place_id's

La Chouette Vineyard
Upsidedown Wine
Tualatin Estate Vineyard
Isenhower Cellars


In [33]:
## Detail keys to save:  "geometry"["location"] as location
detail_dict= dict()
reviews_dict = dict()

for place in enumerate(detail_list):
##  make the search request for the details of the place
    response = requests.get(details_url+f"key={gkey}&place_id={place}").json()["result"]
    detail_dict[place] = dict()
    
## Save all the information    
    for key in response.keys():
        if key != 'geometry':
            if key != "reviews":
                detail_dict[place][key] = response[key]
            else:
                for review in response[key]:
                    reviews_dict[place] = dict()
                    for key2 in review:
                        reviews_dict[place][key2] = review[key2]
        else: 
            detail_dict[place]["location"] = response[key]['location']


In [34]:
pd.DataFrame(detail_dict)

,ChIJ47fROEcPwFQRmvkED1tgYu8,ChIJp3VeV5cPllQRYZuQcpsyd9o,ChIJk7IwOZUdlVQRNfp1VBTEvjY,ChIJqXPayyZColQRvhXcUAe5f34
address_components,"[{'long_name': 'A', 'short_name': 'A', 'types'...","[{'long_name': '2181', 'short_name': '2181', '...","[{'long_name': '10850', 'short_name': '10850',...","[{'long_name': '3471', 'short_name': '3471', '..."
adr_address,"<span class=""street-address"">189 Ankeny Hill R...","<span class=""street-address"">311 Oak St #2181<...","7703, <span class=""street-address"">10850 NW Se...","<span class=""street-address"">3471 Pranger Rd</..."
business_status,OPERATIONAL,OPERATIONAL,OPERATIONAL,OPERATIONAL
delivery,False,NaN,NaN,NaN
formatted_address,"189 Ankeny Hill Rd SE A, Jefferson, OR 97352, USA","311 Oak St #2181, Hood River, OR 97031, USA","7703, 10850 NW Seavey Rd, Forest Grove, OR 971...","3471 Pranger Rd, Walla Walla, WA 99362, USA"
formatted_phone_number,(503) 990-4974,(541) 436-3851,(503) 357-5005,(509) 526-7896
location,"{'lat': 44.7958568, 'lng': -123.0618305}","{'lat': 45.7086516, 'lng': -121.5139728}","{'lat': 45.600534, 'lng': -123.171698}","{'lat': 46.022924, 'lng': -118.3600492}"
icon,https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...
icon_background_color,#7B9EB0,#7B9EB0,#7B9EB0,#7B9EB0
icon_mask_base_uri,https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...,https://maps.gstatic.com/mapfiles/place_api/ic...


In [75]:
fun = pd.DataFrame(detail_response_dict).transpose()
fun.reset_index(inplace=True)
[name for name in fun.columns]

['index',
 'address_components',
 'adr_address',
 'business_status',
 'current_opening_hours',
 'formatted_address',
 'formatted_phone_number',
 'location',
 'icon',
 'icon_background_color',
 'icon_mask_base_uri',
 'international_phone_number',
 'name',
 'opening_hours',
 'photos',
 'place_id',
 'plus_code',
 'rating',
 'reference',
 'reviews',
 'types',
 'url',
 'user_ratings_total',
 'utc_offset',
 'vicinity',
 'website',
 'curbside_pickup']

In [76]:
fun = fun.drop("index", axis=1)
fun

,address_components,adr_address,business_status,current_opening_hours,formatted_address,formatted_phone_number,location,icon,icon_background_color,icon_mask_base_uri,...,rating,reference,reviews,types,url,user_ratings_total,utc_offset,vicinity,website,curbside_pickup
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'long_name': '2181', 'short_name': '2181', '...","<span class=""street-address"">311 Oak St #2181<...",OPERATIONAL,"{'open_now': False, 'periods': [{'close': {'da...","311 Oak St #2181, Hood River, OR 97031, USA",(541) 436-3851,"{'lat': 45.7086516, 'lng': -121.5139728}",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,...,4.9,ChIJp3VeV5cPllQRYZuQcpsyd9o,"[{'author_name': 'Shawn S', 'author_url': 'htt...","[point_of_interest, food, establishment]",https://maps.google.com/?cid=15742106665720257377,29,-420,"311 Oak Street #2181, Hood River",NaN,NaN
2,"[{'long_name': '10850', 'short_name': '10850',...","7703, <span class=""street-address"">10850 NW Se...",OPERATIONAL,"{'open_now': False, 'periods': [{'close': {'da...","7703, 10850 NW Seavey Rd, Forest Grove, OR 971...",(503) 357-5005,"{'lat': 45.600534, 'lng': -123.171698}",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,...,4.8,ChIJk7IwOZUdlVQRNfp1VBTEvjY,"[{'author_name': 'stuart smith (shsmith)', 'au...","[tourist_attraction, point_of_interest, food, ...",https://maps.google.com/?cid=3944805915218541109,104,-420,"7703, 10850 Northwest Seavey Road, Forest Grove",https://www.wvv.com/Tualatin,NaN
3,"[{'long_name': '3471', 'short_name': '3471', '...","<span class=""street-address"">3471 Pranger Rd</...",OPERATIONAL,"{'open_now': False, 'periods': [{'close': {'da...","3471 Pranger Rd, Walla Walla, WA 99362, USA",(509) 526-7896,"{'lat': 46.022924, 'lng': -118.3600492}",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,...,4.9,ChIJqXPayyZColQRvhXcUAe5f34,"[{'author_name': 'Frank Strobel', 'author_url'...","[food, point_of_interest, establishment]",https://maps.google.com/?cid=9115207611893683646,36,-420,"3471 Pranger Road, Walla Walla",http://www.isenhowercellars.com/,True


In [63]:
spam = pd.DataFrame(reviews_dict).transpose().reset_index()
spam["place_id"] = spam["index"]
spam = spam.drop("index", axis=1)
spam

,author_name,author_url,language,original_language,profile_photo_url,rating,relative_time_description,text,time,translated,place_id
0,Alissa Rubin,https://www.google.com/maps/contrib/1124326307...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9e6...,5,5 years ago,"Beautiful property and house, and very accommo...",1503273934,False,ChIJ47fROEcPwFQRmvkED1tgYu8
1,Patrick Hoonan,https://www.google.com/maps/contrib/1000413532...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu9av...,5,3 years ago,Some of the best reds (and rose) in all of WA ...,1566086685,False,ChIJp3VeV5cPllQRYZuQcpsyd9o
2,Celia Asher,https://www.google.com/maps/contrib/1064248255...,en,en,https://lh3.googleusercontent.com/a/ALm5wu2lPx...,5,a year ago,Lori was extremely thorough & knowledgeable & ...,1633131391,False,ChIJk7IwOZUdlVQRNfp1VBTEvjY
3,Paige Stinebaugh,https://www.google.com/maps/contrib/1034529696...,en,en,https://lh3.googleusercontent.com/a-/ACNPEu8le...,5,2 years ago,As locals we have had Isenhower Cellars as a m...,1598333717,False,ChIJqXPayyZColQRvhXcUAe5f34


In [71]:
egg = spam.columns.to_list()
egg.reverse()
print(egg)

['place_id',
 'translated',
 'time',
 'text',
 'relative_time_description',
 'rating',
 'profile_photo_url',
 'original_language',
 'language',
 'author_url',
 'author_name',
 'index']

In [73]:
spam = spam[egg]
spam

,place_id,translated,time,text,relative_time_description,rating,profile_photo_url,original_language,language,author_url,author_name,index
0,ChIJ47fROEcPwFQRmvkED1tgYu8,False,1503273934,"Beautiful property and house, and very accommo...",5 years ago,5,https://lh3.googleusercontent.com/a-/ACNPEu9e6...,en,en,https://www.google.com/maps/contrib/1124326307...,Alissa Rubin,ChIJ47fROEcPwFQRmvkED1tgYu8
1,ChIJp3VeV5cPllQRYZuQcpsyd9o,False,1566086685,Some of the best reds (and rose) in all of WA ...,3 years ago,5,https://lh3.googleusercontent.com/a-/ACNPEu9av...,en,en,https://www.google.com/maps/contrib/1000413532...,Patrick Hoonan,ChIJp3VeV5cPllQRYZuQcpsyd9o
2,ChIJk7IwOZUdlVQRNfp1VBTEvjY,False,1633131391,Lori was extremely thorough & knowledgeable & ...,a year ago,5,https://lh3.googleusercontent.com/a/ALm5wu2lPx...,en,en,https://www.google.com/maps/contrib/1064248255...,Celia Asher,ChIJk7IwOZUdlVQRNfp1VBTEvjY
3,ChIJqXPayyZColQRvhXcUAe5f34,False,1598333717,As locals we have had Isenhower Cellars as a m...,2 years ago,5,https://lh3.googleusercontent.com/a-/ACNPEu8le...,en,en,https://www.google.com/maps/contrib/1034529696...,Paige Stinebaugh,ChIJqXPayyZColQRvhXcUAe5f34


### Ready to pull the detailed information and reviews about each vinyard in the search results

In [ ]:
# Looping through the locations and performing a search for each one, grabbing the pagetokens and place_ids

## Global Variables requied for the loops
saved_place_ids = set()


for i, area in enumerate(areas_to_search):
    payload["location"] = area
    more_results = True
    page_token_list = []

## Looping Through all result pages of one search grabing the next_page_tokens and place_id's
    while more_results:    
        if len(page_token_list) == 0:
            response = requests.get(nearby_url, payload).json() 
        else:
            url = nearby_url+ f"key={gkey}&pagetoken={page_token_list[-1]}"
            response = requests.get(url).json() 

        for place in response["results"]:
            saved_place_ids.add(place["place_id"])
        try:
            page_token_list.append(response["next_page_token"])
        except:
            more_results = False 
            

        sleep(2)
    
    print(f"{i}. End for this location, moving onto the next. \nNew Totals:")    
    print("Number of page_tokens saved this time:", len(page_token_list))
    print("Number of unique place_ids saved:", len(saved_place_ids))
    print("==============\n===============\n===============")

## radius:20000 = 405 results    

0. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 36
1. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 49
2. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 50
3. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 50
4. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 1
Number of unique place_ids saved: 58
5. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 113
6. End for this location, moving onto the next. 
New Totals:
Number of page_tokens saved this time: 2
Number of unique place_ids saved: 171
7. End for this location,

In [ ]:
## Detail keys to save:  "geometry"["location"] as location
detail_dict= dict()
reviews_dict = dict()

for place in enumerate(saved_place_ids):
##  make the search request for the details of the place
    response = requests.get(details_url+f"key={gkey}&place_id={place}").json()["result"]
    detail_dict[place] = dict()
    
## Save all the information    
    for key in response.keys():
        if key != 'geometry':
            if key != "reviews":
                detail_dict[place][key] = response[key]
            else:
                for review in response[key]:
                    reviews_dict[place] = dict()
                    for key2 in review:
                        reviews_dict[place][key2] = review[key2]
        else: 
            detail_dict[place]["location"] = response[key]['location']


In [ ]:
details = pd.DataFrame(detail_response_dict).transpose()
details.reset_index(inplace=True)
details

In [ ]:
spam = pd.DataFrame(reviews_dict).transpose().reset_index()
spam["place_id"] = spam["index"]
spam = spam.drop("index", axis=1)
spam